In [117]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import sklearn
import numpy as np


# Récuperation du json et conversion en quelque chose d'utilisable

In [118]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [119]:
reconstitution = from_json(list[Voiture],contenu_fichier)



In [120]:
reconstitution[0]

Voiture(modele='308', carburant='Diesel', prix=20290, kilometrage=75967, garantie_kilometrage='non garanti', boite_de_vitesse='Automatique', transmission=2, couleur='Blanc', garantie='SPOTICAR PREMIUM', date_mise_circulation=2019, puissance=130, silhouette='Berline', nb_places=5, utilisation_prec='Ex-Loueur', puissance_fiscale=6, critair=2, ptac=1890, nb_portes=5)

# Conversion en base de données panda et nettoyage

In [121]:
import pandas as pd

In [122]:
df = pd.DataFrame(reconstitution)


In [123]:
df.describe()

,prix,kilometrage,date_mise_circulation
count,600.000000,600.000000,600.000000
mean,26750.178333,28932.145000,2020.473333
std,8835.590982,29263.082939,1.738100
min,7490.000000,3.000000,2011.000000
25%,19721.750000,7000.000000,2019.000000
50%,26240.000000,19521.000000,2021.000000
75%,33524.750000,41282.500000,2022.000000
max,59980.000000,176600.000000,2022.000000


In [124]:
df.head()


,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,308,Diesel,20290,75967,non garanti,Automatique,2,Blanc,SPOTICAR PREMIUM,2019,130,Berline,5,Ex-Loueur,6,2,1890,5
1,3008,Diesel,31900,29066,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,2,NA,5
2,3008,Essence,33920,31997,non garanti,Automatique,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,1,NA,5
3,2008,Essence,21980,40952,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,NA,5,1,NA,5
4,2008,Essence,22990,7901,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,Ex-Particulier,5,1,NA,5


In [125]:
for i in df.columns:
    col_selec = f"df.{i}.unique()"
    
    print(f"{i} : {eval(col_selec)} ")
# retourne toutes les modalités pour chaque variable

modele : ['308' '3008' '2008' '208' '108' '308 SW' '5008' '508' 'RIFTER' 'BOXER'
 '508 SW' 'TRAVELLER' 'PARTNER' 'EXPERT' '4008'] 
carburant : ['Diesel' 'Essence' 'Electrique' 'Hybride rechargeable'] 
prix : [20290 31900 33920 21980 22990 21990 34390 17890 32990 20490 18990 11990
 26290 33490 17490 32490 25990 26590 22490 17690 34490 36990 38790 32590
 28990 25290 16990 22190 23980 33790 38490 22390 26990 29990 38980 31690
 24590 35990 38290 28790 26190 23790 26299 21490 31290 35690 22290 46790
 15490 28980 33091 30790 40490 28490 36390 17990 39790 23220 59980 38990
 24790 23290 32480 20480 34990 27790 23990 15480 23390 52890 29290 27590
 27990 26490 15290 27690 39990 42580 36490 32390 28690 32300 24990 19290
 42590 46290 31580 52190 37990 28590 39390 39690 38280 24490 23150 26390
 37180 21590 35299 38499 37099 24199 42199 14790 14799 37490 39631 47490
 37599 14590 33990 28390 16490 18480 17980 22980 39290 22180 27980 12480
 19280 33480 12280 22280 33980 31780 40980 24280 15380 18580 3

## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

## Création de dummys


In [126]:
df = conversion_df.data_frame_pandas('donnees.json')

In [127]:
df.dtypes


modele                    object
carburant                 object
prix                       int64
kilometrage                int64
garantie_kilometrage      object
boite_de_vitesse          object
transmission             float64
couleur                   object
garantie                  object
date_mise_circulation      int64
puissance                float64
silhouette                object
nb_places                float64
utilisation_prec          object
puissance_fiscale        float64
critair                  float64
ptac                     float64
nb_portes                float64
dtype: object

In [128]:
df.ptac

0      1890.000000
1      1874.729367
2      1874.729367
3      1874.729367
4      1874.729367
          ...     
595    1940.000000
596    1770.000000
597    2000.000000
598    1874.729367
599    2790.000000
Name: ptac, Length: 600, dtype: float64

In [129]:
df.convert_dtypes().dtypes

modele                    string
carburant                 string
prix                       Int64
kilometrage                Int64
garantie_kilometrage      string
boite_de_vitesse          string
transmission             Float64
couleur                   string
garantie                  string
date_mise_circulation      Int64
puissance                Float64
silhouette                string
nb_places                Float64
utilisation_prec          string
puissance_fiscale        Float64
critair                  Float64
ptac                     Float64
nb_portes                Float64
dtype: object

## Passage sous numpy + ML

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
import conversion_df

reload(conversion_df)

<module 'conversion_df' from 'c:\\Users\\mathi\\Documents\\Cours\\M2\\Machine Learning\\VERON_NEPVEUX_PROJECT\\veron_nepveux_project\\conversion_df.py'>

In [131]:
X, y = conversion_df.data_frame_modele('donnees.json')

In [132]:
X

array([[7.5967e+04, 2.0190e+03, 1.3000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.9066e+04, 2.0210e+03, 1.3000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.1997e+04, 2.0210e+03, 1.3000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [2.1164e+04, 2.0220e+03, 1.3100e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0000e+01, 2.0220e+03, 1.4500e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0000e+01, 2.0220e+03, 1.7700e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [133]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [134]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
en_df.head()
        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.436e+08, tolerance: 2.699e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.475e+08, tolerance: 2.967e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017353,0.001620,0.000000,0.000000,0.015625,0.01,"{'alpha': 0.015625, 'l1_ratio': 0.01}",0.931773,0.909541,0.921335,0.934228,0.915186,0.922413,0.009447,8
1,0.014362,0.001493,0.000200,0.000399,0.015625,0.25,"{'alpha': 0.015625, 'l1_ratio': 0.25}",0.931670,0.909574,0.921492,0.933577,0.916206,0.922504,0.009105,7
2,0.020543,0.002570,0.000399,0.000488,0.015625,0.5,"{'alpha': 0.015625, 'l1_ratio': 0.5}",0.931117,0.909489,0.921478,0.932954,0.917505,0.922509,0.008704,5
3,0.034707,0.004009,0.000399,0.000489,0.015625,0.75,"{'alpha': 0.015625, 'l1_ratio': 0.75}",0.929390,0.909147,0.921198,0.933079,0.919150,0.922393,0.008373,11
4,0.051465,0.038298,0.000000,0.000000,0.015625,1,"{'alpha': 0.015625, 'l1_ratio': 1}",0.926210,0.906920,0.924817,0.941423,0.917169,0.923308,0.011359,1


In [135]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 0.015625, 'l1_ratio': 1}, 0.9233077299957747)

In [136]:
from sklearn.neighbors import KNeighborsRegressor

KNN

In [137]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [138]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000398,0.000488,0.008778,0.001467,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.494067,0.453842,0.432240,0.587952,0.486897,0.491000,0.053403,11
1,0.000997,0.000632,0.011016,0.001025,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.523950,0.517758,0.492073,0.641959,0.566547,0.548457,0.052521,1
2,0.000000,0.000000,0.007982,0.000631,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.468571,0.443095,0.386004,0.581147,0.477782,0.471320,0.063540,12
3,0.000598,0.000488,0.008777,0.001323,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.513022,0.515401,0.465952,0.644959,0.563534,0.540573,0.060637,2
4,0.000797,0.000399,0.006783,0.000746,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.449932,0.420647,0.381347,0.563723,0.441715,0.451473,0.060935,14


In [139]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 5, 'weights': 'distance'}, 0.5484572926127578)

In [140]:
from sklearn.ensemble import RandomForestRegressor

In [141]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256),
    }
)

In [142]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.025133,0.002705,0.001002,0.000009,8,{'n_estimators': 8},0.849297,0.916370,0.880615,0.912923,0.863324,0.884506,0.026557,6
1,0.050878,0.002279,0.001577,0.000772,16,{'n_estimators': 16},0.835263,0.920619,0.877547,0.928286,0.872412,0.886826,0.034098,4
2,0.110691,0.011127,0.003010,0.000627,32,{'n_estimators': 32},0.839848,0.921811,0.868258,0.930395,0.872537,0.886570,0.034286,5
3,0.188696,0.009290,0.004350,0.000451,64,{'n_estimators': 64},0.844031,0.925591,0.864014,0.938529,0.871117,0.888656,0.036763,3
4,0.383240,0.021314,0.008585,0.000481,128,{'n_estimators': 128},0.843311,0.924525,0.867993,0.939612,0.873027,0.889693,0.036346,2


In [143]:
rfr_gs.best_params_, rfr_gs.best_score_

({'n_estimators': 256}, 0.8931274764122259)

In [144]:
from sklearn.svm import SVR

In [145]:
svr = SVR()
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "epsilon": (0.1, 1.0, 10),
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010337,0.000730,0.005820,0.001596,0.1,0.1,"{'C': 0.1, 'epsilon': 0.1}",-0.008121,-0.000357,-0.023442,-0.004581,-0.068792,-0.021058,0.025106,9
1,0.009565,0.000496,0.004602,0.000491,0.1,1.0,"{'C': 0.1, 'epsilon': 1.0}",-0.008121,-0.000356,-0.023442,-0.004578,-0.068792,-0.021058,0.025107,8
2,0.008577,0.000502,0.004388,0.000489,0.1,10,"{'C': 0.1, 'epsilon': 10}",-0.008121,-0.000416,-0.023442,-0.004438,-0.068792,-0.021042,0.025115,7
3,0.008767,0.000378,0.003987,0.000007,1.0,0.1,"{'C': 1.0, 'epsilon': 0.1}",-0.004013,0.003434,-0.018785,0.000536,-0.066298,-0.017025,0.025796,5
4,0.008786,0.000400,0.004379,0.000497,1.0,1.0,"{'C': 1.0, 'epsilon': 1.0}",-0.004013,0.003434,-0.018785,0.000536,-0.066298,-0.017025,0.025796,5


In [146]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 0.1}, 0.024925707301698474)

In [147]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [148]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [149]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [150]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010987,0.001410,0.004785,0.000731,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.008303,-0.000492,-0.023633,-0.004830,-0.068732,-0.021198,0.025013,9
1,0.009589,0.000500,0.004182,0.000385,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.008303,-0.000494,-0.023633,-0.004822,-0.068732,-0.021197,0.025014,8
2,0.009380,0.001362,0.004583,0.000485,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.008303,-0.000554,-0.023633,-0.004678,-0.068732,-0.021180,0.025023,7
3,0.009552,0.000503,0.005590,0.000492,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.005810,0.002099,-0.020682,-0.001944,-0.065684,-0.018404,0.024862,6
4,0.009387,0.000479,0.004577,0.000480,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.005810,0.002099,-0.020682,-0.001943,-0.065684,-0.018404,0.024862,5


In [151]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 10},
 0.009145706160399225)

In [152]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [153]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [154]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [155]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.265284,0.006475,0.000399,0.000488,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-6.371930,-9.413385,-10.806891,-8.847154,-9.456761,-8.979224,1.454120,22
1,0.492616,0.029562,0.000399,0.000488,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-6.320893,-9.387310,-10.720110,-8.793714,-9.387221,-8.921850,1.445570,17
2,0.604582,0.011835,0.000804,0.000402,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-6.260621,-9.223860,-10.604714,-8.666408,-9.310104,-8.813141,1.425643,11
3,0.163496,0.005173,0.000991,0.000013,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-5.881779,-8.492671,-10.359269,-8.088452,-8.686230,-8.301680,1.436622,6
4,0.268477,0.007605,0.000198,0.000395,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-6.366901,-9.422666,-10.772455,-8.854116,-9.474903,-8.978208,1.448992,21


In [156]:
pln_gs.best_params_

pln_gs.best_score_

-8.050881234562514

In [157]:
pln_gs.best_params_

{'neurones__alpha': 1e-06, 'neurones__hidden_layer_sizes': (20, 20)}

In [158]:
essai = en_gs.best_estimator_

In [159]:
prix_prédits = essai.predict(X)

In [160]:
prix_reels = y

In [161]:
prix_reels,prix_prédits

(array([20290, 31900, 33920, 21980, 22990, 21990, 34390, 17890, 32990,
        20490, 18990, 11990, 26290, 33490, 17490, 32490, 25990, 26590,
        26290, 22490, 17690, 34490, 36990, 38790, 18990, 32590, 28990,
        25290, 16990, 22190, 23980, 33790, 38490, 22390, 22490, 26990,
        29990, 38980, 25990, 31690, 24590, 35990, 38290, 28790, 32490,
        26190, 34490, 23790, 26299, 38290, 35990, 21490, 25990, 16990,
        31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990, 33091,
        30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490, 23220,
        22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990, 24590,
        27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390, 34490,
        52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290, 35690,
        27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990, 28690,
        32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290, 31580,
        52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690, 26990,
      

In [162]:
len(prix_prédits)

600

In [163]:
len(prix_reels)

600

In [164]:
prix_prédits

array([21037.86247711, 30276.13931246, 30087.99496474, 20720.6902187 ,
       24219.18811635, 22881.93035411, 34911.2432658 , 18567.38496696,
       33837.93300547, 18303.64431675, 20030.39942208, 13749.90766034,
       26574.27192306, 33963.36711385, 17331.57327316, 31605.43343999,
       25823.50124032, 25993.13139676, 30016.36168973, 21212.42877819,
       18195.90129318, 34911.8442702 , 38210.14590932, 35789.62109964,
       19857.3385501 , 32617.05085461, 30113.70606557, 27229.33058545,
       16905.83798747, 23488.35714219, 24507.67497479, 32380.32568231,
       36375.79066291, 22394.72518269, 22601.20085944, 29745.87901137,
       27536.37795691, 36120.44814934, 27590.67302229, 29513.80596491,
       25813.15438974, 34065.67582121, 36914.66780641, 32614.3042414 ,
       31658.8654705 , 27170.69768025, 34089.81979117, 24297.39388954,
       26792.04754076, 36423.81670242, 37927.39308826, 20107.59715134,
       28468.88389092, 16803.02434698, 31278.4800103 , 33838.05279206,
      

In [165]:
prix_reels

array([20290, 31900, 33920, 21980, 22990, 21990, 34390, 17890, 32990,
       20490, 18990, 11990, 26290, 33490, 17490, 32490, 25990, 26590,
       26290, 22490, 17690, 34490, 36990, 38790, 18990, 32590, 28990,
       25290, 16990, 22190, 23980, 33790, 38490, 22390, 22490, 26990,
       29990, 38980, 25990, 31690, 24590, 35990, 38290, 28790, 32490,
       26190, 34490, 23790, 26299, 38290, 35990, 21490, 25990, 16990,
       31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990, 33091,
       30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490, 23220,
       22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990, 24590,
       27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390, 34490,
       52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290, 35690,
       27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990, 28690,
       32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290, 31580,
       52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690, 26990,
       40490, 38280,

Objectif : maximiser le surplus de l'acheteur, i.e, si l'acheteur à un budget R, et n contraintes, quel véhicule maximiserait son surplus


In [170]:
data_finale = conversion_df.data_frame_dummies(df)

In [171]:
data_finale['prix_predits'] = prix_prédits
data_finale['surplus_conso'] = data_finale['prix'] - data_finale['prix_predits']

In [172]:
data_finale

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,critair,ptac,nb_portes,prix_predits,...,AUTOEXPERT,SPOTICAR ADVANCED,SPOTICAR ESSENTIAL,SPOTICAR PREMIUM,Ex-Auto-école,Ex-Import,Ex-Loueur,Ex-Particulier,Ex-Société,Véhicule de direction
0,20290,75967,2019,130.0,5.0,6.0,2.0,1890.000000,5.000000,21037.862477,...,0,0,0,1,0,0,1,0,0,0
1,31900,29066,2021,130.0,5.0,7.0,2.0,1874.729367,5.000000,30276.139312,...,0,0,0,1,0,0,1,0,0,0
2,33920,31997,2021,130.0,5.0,7.0,1.0,1874.729367,5.000000,30087.994965,...,0,0,0,1,0,0,1,0,0,0
3,21980,40952,2021,100.0,5.0,5.0,1.0,1874.729367,5.000000,20720.690219,...,0,0,0,1,0,0,1,0,0,0
4,22990,7901,2021,100.0,5.0,5.0,1.0,1874.729367,5.000000,24219.188116,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,33990,3254,2022,131.0,5.0,7.0,2.0,1940.000000,5.000000,32300.883779,...,0,0,0,1,0,0,1,0,0,0
596,31490,2535,2022,131.0,5.0,7.0,2.0,1770.000000,5.000000,31991.823415,...,0,0,0,1,0,1,0,0,0,0
597,37490,21164,2022,131.0,5.0,7.0,2.0,2000.000000,5.000000,35719.627210,...,0,0,0,1,0,0,1,0,0,0
598,36490,10,2022,145.0,3.0,7.0,2.0,1874.729367,4.934673,32783.446821,...,0,0,0,1,0,0,1,0,0,0


In [173]:
newdf =data_finale.loc[
    (data_finale["208"] == 1) & 
    (data_finale["Essence"]) & 
    (data_finale['Gris']) &
    (data_finale["Citadine"]) &
    (data_finale['2']) &
    (data_finale.date_mise_circulation >= 2017) 
    ]

KeyError: '2'

In [ ]:
newdf.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes,prix_predits,surplus_conso


In [ ]:
meilleur_choix = newdf.loc[newdf['surplus_conso'] == newdf['surplus_conso'].max()]

In [ ]:
tri_meilleures = newdf.sort_values(by = 'surplus_conso', ascending = False).head()

In [ ]:
tri_meilleures

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes,prix_predits,surplus_conso


In [ ]:
tri_meilleures.index

Int64Index([], dtype='int64')